In [ ]:
##Load the packages
import scipy.sparse as sp
import pickle
import numpy as np
import scanpy
import pandas as pd
from itertools import chain
import os
import matplotlib.pyplot as plt
from sklearn.neighbors import NearestNeighbors

# The function for preprocessing the data and extracting the silver standard of Cell-cell-communcation

In [ ]:
def Preprocess_Atlas(tissue_dict,tissue_index,
                    cellchat_ligand_receptor_filedir,
                    if_filtercells = False,min_genes = 200,
                    if_filtergenes = False,min_cells = 3,
                    if_HVG = False,N_HVG = 3000,
                    num_spatialneighbor = 10):
    #################################################################################################
    print("tissue_index: " + str(tissue_index + 1) + " Tissue: " + tissue_dict['Tissue'][tissue_index])
    slice_name = tissue_dict['Tissue'][tissue_index]
    #################################################################################################
    
    ##1. Load the spatial transcriptomics data
    #################################################################################################
    if tissue_dict['Technology'][tissue_index] == '10XVisium' or tissue_dict['Technology'][
        tissue_index] == '10XVisiumHD':
        ##Gene expression
        expression_data = tissue_dict['Gene_expression'][tissue_index].X
        ##Cell label and gene label
        genes = (tissue_dict['Gene_expression'][tissue_index].var_names).tolist()
        cells = (tissue_dict['Gene_expression'][tissue_index].obs_names).tolist()
    if tissue_dict['Technology'][tissue_index] == 'MERFISH_Zeng' or tissue_dict['Technology'][
        tissue_index] == 'MERFISH_Zhuang':
        ##Gene expression
        expression_data = tissue_dict['Gene_expression'][tissue_index]['gene_expression']
        ##Cell label and gene label
        genes = tissue_dict['Gene_expression'][tissue_index]['feature_names']['feature_names'].tolist()
        cells = tissue_dict['Gene_expression'][tissue_index]['feature_names']['cellname'].tolist()
    ##Build the adata
    adata = scanpy.AnnData(X=expression_data, obs={'cells': cells}, var={'genes': genes})
    adata.obs_names = cells
    adata.var_names = genes
    del expression_data, cells, genes
    ##Add Tissue name to the adata
    adata.obs["Tissue"] = tissue_dict['Tissue'][tissue_index]
    ##Add Technology to the adata
    adata.obs["Technology"] = tissue_dict['Technology'][tissue_index]
    ##Add spatial location information to the adata
    if tissue_dict['Technology'][tissue_index] == '10XVisium' or tissue_dict['Technology'][
        tissue_index] == '10XVisiumHD':
        x_coord_name = "pxl_row_in_fullres"
        y_coord_name = "pxl_col_in_fullres"
        tissue_dict['Spatial_location'][tissue_index].index = tissue_dict['Spatial_location'][tissue_index]['barcode']
    if tissue_dict['Technology'][tissue_index] == 'MERFISH_Zeng' or tissue_dict['Technology'][
        tissue_index] == 'MERFISH_Zhuang':
        x_coord_name = "x"
        y_coord_name = "y"
    adata.obs["xcoord"] = tissue_dict['Spatial_location'][tissue_index][x_coord_name][adata.obs.index]
    adata.obs["ycoord"] = tissue_dict['Spatial_location'][tissue_index][y_coord_name][adata.obs.index]
    ##Add other meta information to the adata
    if len((tissue_dict['Metadata'][tissue_index])) > 0:
        adata.obs[['donor_label', 'donor_genotype', 'donor_sex', 'neurotransmitter', 'class', 'subclass', 'supertype',
                   'cluster']] = tissue_dict['Metadata'][tissue_index][
            ['donor_label', 'donor_genotype', 'donor_sex', 'neurotransmitter', 'class', 'subclass', 'supertype',
             'cluster']]
    adata.obsm['spatial'] = np.array(tissue_dict['Spatial_location'][tissue_index][[x_coord_name, y_coord_name]])
    ##Data preprocessing
    if if_filtercells:
        scanpy.pp.filter_cells(adata, min_genes=min_genes)
    if if_filtergenes:
        scanpy.pp.filter_genes(adata, min_cells=min_cells)
    adata_raw = adata.copy()
    adata.X_raw = adata.X.copy()
    scanpy.pp.normalize_total(adata, target_sum=1e4)
    scanpy.pp.log1p(adata)
    if if_HVG:
        scanpy.pp.highly_variable_genes(adata, n_top_genes=N_HVG, flavor='seurat_v3')
        adata = adata[:, adata.var.highly_variable]
    #################################################################################################
    
    ##2. Load the CellChat ligand-receptor database
    #################################################################################################
    ##load the Ligand-Receptor pairs from CellChat
    cellchat_ligand_receptor = pd.read_csv(cellchat_ligand_receptor_filedir, header=None)
    cellchat_ligand_receptor = np.array(cellchat_ligand_receptor.iloc[:, 0]).tolist()
    for i in range(len(cellchat_ligand_receptor)):
        cellchat_ligand_receptor[i] = cellchat_ligand_receptor[i].replace(" ", "")
    Ligand_list = [cellchat_ligand_receptor[i].split("-")[0] for i in range(len(cellchat_ligand_receptor))]
    Ligand_list_unique = np.unique(np.array(Ligand_list)).tolist()
    Receptor_list0 = [cellchat_ligand_receptor[i].split("-")[1] for i in range(len(cellchat_ligand_receptor))]
    Receptor_list = []
    for j in range(len(Receptor_list0)):
        if '+' in Receptor_list0[j]:
            Receptor_list.append(Receptor_list0[j].replace("(", "").replace(")", "").split("+"))
        else:
            Receptor_list.append([Receptor_list0[j]])
    LR_in_bool_list = []
    use_Ligand_list = []
    use_Receptor_list = []
    for LR_index in range(len(cellchat_ligand_receptor)):
        ligand_cur = cellchat_ligand_receptor[LR_index].split("-")[0]
        receptor_cur = cellchat_ligand_receptor[LR_index].split("-")[1]
        if "+" in receptor_cur:
            receptor_cur = receptor_cur.replace("(", "").replace(")", "").split("+")
        # LR_in_bool = (np.sum(np.isin(adata.var_names,receptor_cur)) == len(receptor_cur)) *(np.sum(np.isin(adata.var_names,ligand_cur)) == len(ligand_cur))
        LR_in_bool = (np.sum(~np.isin(receptor_cur,adata.var_names)) == 0) * (
                np.sum(np.isin(adata.var_names, ligand_cur)) > 0)
        # LR_in_bool = (np.sum(np.isin(adata.var_names, receptor_cur)) == len(receptor_cur)) * (
        #         np.sum(np.isin(adata.var_names, ligand_cur)) > 0)
        LR_in_bool_list.append(LR_in_bool)
        if LR_in_bool:
            if isinstance(receptor_cur, list):
                receptor_cur = np.array(receptor_cur)[np.where(np.isin(receptor_cur, adata.var_names))[0]].tolist()
            use_Ligand_list.append(ligand_cur)
            use_Receptor_list.append(receptor_cur)
    Ligand_to_Receptor_list = [{'Ligand': use_Ligand_list[i],
                                'Receptor': use_Receptor_list[i]} for i in range(len(use_Ligand_list))]
    #################################################################################################
    
    ##3. Construct the ligand-receptor interaction matrix of the spatial transcriptomics data
    #################################################################################################
    cell_spatial = adata.obsm['spatial']
    spatialnn_use = NearestNeighbors(n_neighbors=num_spatialneighbor + 1, algorithm='auto')
    spatialnn_use.fit(cell_spatial)
    spatialdist, spatial_neighborindex = spatialnn_use.kneighbors(cell_spatial)
    ##
    index_mat = np.tile(np.arange(spatial_neighborindex.shape[0])[:, np.newaxis], (1, spatial_neighborindex.shape[1]))
    ##Random permutation
    np.random.seed(0)
    random_permutation_num = 100
    # random_permutation_num = 1
    index_mat_randomperm_list = []
    spatial_neighborindex_randomperm_list = []
    for i in range(random_permutation_num):
        random_index = np.random.permutation(np.arange(cell_spatial.shape[0]))
        cell_spatial_randomperm = cell_spatial[random_index, :]
        spatialnn_use_randomperm = NearestNeighbors(n_neighbors=num_spatialneighbor + 1, algorithm='auto')
        spatialnn_use_randomperm.fit(cell_spatial_randomperm)
        spatialdist_randomperm, spatial_neighborindex_randomperm = spatialnn_use_randomperm.kneighbors(cell_spatial_randomperm)
        ##
        index_mat_randomperm = np.tile(np.arange(spatial_neighborindex_randomperm.shape[0])[:, np.newaxis],
                            (1, spatial_neighborindex_randomperm.shape[1]))
        spatial_neighborindex_randomperm_list.append(spatial_neighborindex_randomperm)
        index_mat_randomperm_list.append(index_mat_randomperm)
    ##
    LRI_intensity_Cellbycell_list = []
    for LR_index in range(len(Ligand_to_Receptor_list)):
        # print("The LR pair: " + str(LR_index))
        Ligand_cur = Ligand_to_Receptor_list[LR_index]['Ligand']
        Receptor_cur = Ligand_to_Receptor_list[LR_index]['Receptor']
        Receptor_cur = np.array(Receptor_cur)[(np.isin(Receptor_cur, adata.var_names))].tolist()
        Ligand_index_inusegene = np.where(np.isin(adata.var_names, Ligand_cur))[0]
        Receptor_index_inusegene = np.where(np.isin(adata.var_names, Receptor_cur))[0]
        ##
        Ligand_expression = np.array(adata.X[:, Ligand_index_inusegene].todense())
        Receptor_expression = np.array(np.max(adata.X[:, Receptor_index_inusegene].todense(), axis=1))
        LRI_intensity = np.power(Ligand_expression[index_mat[:,1:].ravel(),:] *
                                 Receptor_expression[spatial_neighborindex[:,1:].ravel()],0.5)
        ##Calculate the null distribution
        LRI_intensity_randperm = np.power(Ligand_expression[index_mat_randomperm_list[0][:,1:].ravel(),:] *
                                 Receptor_expression[spatial_neighborindex_randomperm_list[0][:,1:].ravel()],0.5)
        threshold_perm = np.quantile(LRI_intensity_randperm,0.99)
        if np.sum(LRI_intensity) > 0:
            LRI_intensity = LRI_intensity.ravel()
            sender_index = index_mat[:,1:].ravel()
            reciever_index = spatial_neighborindex[:,1:].ravel()
            LRI_intensity = LRI_intensity
            dict_cur = {'LRpair': Ligand_to_Receptor_list[LR_index],
                                                  'LRI_intensity': LRI_intensity,
                                                  "If_significant_interaction": LRI_intensity>threshold_perm,
                                                  'sender_index': sender_index,
                                                  'sender_cell': adata.obs.index[sender_index],
                                                  'reciever_index': reciever_index,
                                                  'reciever_cell': adata.obs.index[reciever_index]}
            LRI_intensity_Cellbycell_list.append(dict_cur)
            trans_value = []
            for i in range(len(dict_cur["If_significant_interaction"])):
                if dict_cur["If_significant_interaction"][i]:
                    trans_value.append(1)
                else:
                    trans_value.append(-1)
            adj_matrix_sparse = sp.csr_matrix((trans_value, (sender_index, reciever_index)),
                                              shape=(adata.shape[0], adata.shape[0]))
            ##Add to the adata
            if isinstance(Ligand_to_Receptor_list[LR_index]['Receptor'],list):
                LR_name = Ligand_to_Receptor_list[LR_index]['Ligand']+ " -> " + Ligand_to_Receptor_list[LR_index]['Receptor'][0] + "+" + Ligand_to_Receptor_list[LR_index]['Receptor'][1]
            else:
                LR_name = Ligand_to_Receptor_list[LR_index]['Ligand'] + " -> " + \
                          Ligand_to_Receptor_list[LR_index]['Receptor']
            adata_raw.obsp[LR_name] = adj_matrix_sparse
    print("The LR pair in data: " + str(len(LRI_intensity_Cellbycell_list)))
    # print("The number of LR edge: "+ str(np.sum([len(LRI_intensity_Cellbycell_list[i]['LRI_intensity']) for i in range(len(LRI_intensity_Cellbycell_list))])))
    ##Without self
    spatial_neighborindex_withoutself = spatial_neighborindex[:, 1:]
    index_mat_withoutself = np.tile(np.arange(spatial_neighborindex_withoutself.shape[0])[:, np.newaxis], (1, spatial_neighborindex_withoutself.shape[1]))
    spatial_neighbor_index_withoutself = np.stack((index_mat_withoutself.ravel(),spatial_neighborindex_withoutself.ravel()))
    print("#################################################################################################")
    #################################################################################################
    ##4. Return the adata and the ligand-receptor interaction matrix
    #################################################################################################
    return adata_raw,  LRI_intensity_Cellbycell_list,slice_name

##mkdir
def mkdir(path):
    folder = os.path.exists(path)
    if not folder:
        os.makedirs(path)
    return path

# Set up the ditrectory of the loading data and the saving objects

In [ ]:
filedir_main = "D:/Project_genesllm/Data/"

# Obtain the slices information of the collected spatial transcriptomics data

In [ ]:
##All the slices
with open(filedir_main + "tissue_dict_MouseBrain.pickle", 'rb') as pickle_file:
    tissue_dict = pickle.load(pickle_file)
################################################################################################
print("The summary of tissue_dict_MouseBrain:")
print(tissue_dict.keys())
print(np.unique(tissue_dict['Technology']))
num_tissue = len(tissue_dict['Tissue'])
print("The number of tissues: " + str(num_tissue))
##Merfish slices index
Merfish_Zeng_index = np.where(np.array(tissue_dict['Technology']) == 'MERFISH_Zeng')[0]
Merfish_Zhuang_index = np.where(np.array(tissue_dict['Technology']) == 'MERFISH_Zhuang')[0]
print("#################################################################################################")

# Extract the silver standard of Cell-cell-communication slices by slices

In [ ]:
print("The MERFISH slices in Zeng:" + str(len(Merfish_Zeng_index)))
print("The MERFISH slices in Zhuang:" + str(len(Merfish_Zhuang_index)))
##
tissue_index_type_count = 0
for tissue_index_type in [Merfish_Zeng_index,Merfish_Zhuang_index]:
    tissue_index_type_count = tissue_index_type_count + 1
    for tissue_index_use in tissue_index_type:
            adata_raw, LRI_intensity_Cellbycell_list, slice_name = Preprocess_Atlas(tissue_dict=tissue_dict,
                                                                           tissue_index=tissue_index_use,
                                                                           cellchat_ligand_receptor_filedir = filedir_main + "Mouse_LRI_Cellchat.txt",
                                                                           if_filtercells=False, min_genes=200,
                                                                           if_filtergenes=False, min_cells=3,
                                                                           if_HVG=False, N_HVG=3000,
                                                                           num_spatialneighbor=10)
            if tissue_index_type_count == 1:
                mkdir(filedir_main + "CCC_adata_Zeng_mousebrain550/")
                adata_raw.write(filedir_main + "CCC_adata_Zeng_mousebrain550/" + slice_name + ".h5ad")
            else:
                mkdir(filedir_main + "CCC_adata_MERFISH_mousebrain1122/")
                adata_raw.write(filedir_main + "CCC_adata_MERFISH_mousebrain1122/" + slice_name + ".h5ad")
            ##
            del adata_raw, LRI_intensity_Cellbycell_list, slice_name